In [ ]:
import pandas as pd
import numpy as np

In [ ]:
match = pd.read_csv('matches.csv')

In [ ]:
delivery = pd.read_csv('deliveries.csv')

In [ ]:
match.head()

In [ ]:
match.shape

In [ ]:
delivery.head()

In [ ]:
delivery.shape

In [ ]:
total_score_df=delivery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()

In [ ]:
total_score_df

In [ ]:
 total_score_df[total_score_df['inning']== 1]

In [ ]:
total_score_df= total_score_df[total_score_df['inning']== 1]

In [ ]:
match_df=match.merge(total_score_df[['match_id','total_runs']],left_on='id',right_on='match_id')

In [ ]:
match_df

In [ ]:
match_df.shape

In [ ]:
match_df['team1'].unique()

In [ ]:
team=[ 'Sunrisers Hyderabad',
       'Mumbai Indians', 
       'Kolkata Knight Riders',
       'Kings XI Punjab',
       'Chennai Super Kings',
       'Delhi Capitals',
       'Rajasthan Royals'
    
]

In [ ]:
match_df['team1']=match_df['team1'].str.replace('Delhi Deredevils','Delhi Capitals')
match_df['team2']=match_df['team2'].str.replace('Delhi Deredevils','Delhi Capitals')

match_df['team1']=match_df['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match_df['team2']=match_df['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')

In [ ]:
match_df=match_df[match_df['team1'].isin(team)]
match_df=match_df[match_df['team2'].isin(team)]

In [ ]:
match_df.shape

In [ ]:
match_df['dl_applied'].value_counts()

In [ ]:
match_df = match_df[match_df['dl_applied']==0]

In [ ]:
match_df

In [ ]:
match_df=match_df[['match_id','city','winner','total_runs']]

In [ ]:
match_df.shape

In [ ]:
match_df.merge(delivery,on='match_id')

In [ ]:
delivery_df=match_df.merge(delivery,on='match_id')

In [ ]:
delivery_df=delivery_df[delivery_df['inning']==2]

In [ ]:
delivery_df.shape

In [ ]:
delivery_df

In [ ]:
delivery_df['current_score']=delivery_df.groupby('match_id').cumsum()['total_runs_y']

In [ ]:
delivery_df

In [ ]:
delivery_df['runs_left']=delivery_df['total_runs_x']-delivery_df['current_score']

In [ ]:
delivery_df

In [ ]:
126 - (delivery_df['over']*6 + delivery_df['ball'])

In [ ]:
delivery_df['balls_left']=126 - (delivery_df['over']*6 + delivery_df['ball'])

In [ ]:
delivery_df

In [ ]:
delivery_df['player_dismissed']=delivery_df['player_dismissed'].fillna('0')
delivery_df['player_dismissed']=delivery_df['player_dismissed'].apply(lambda x:x if x ==  '0' else '1')
delivery_df['player_dismissed']=delivery_df['player_dismissed'].astype('int')
wickets = delivery_df.groupby('match_id').cumsum()['player_dismissed'].values
delivery_df['wickets']=10 -wickets
delivery_df.head()

In [ ]:
delivery_df.tail()

In [ ]:
# crr = runs/overs
delivery_df['crr']=(delivery_df['current_score']*6)/(120-delivery_df['balls_left'])

In [ ]:
delivery_df.head()

In [ ]:
#rrr 
delivery_df['rrr']=(delivery_df['runs_left']*6)/delivery_df['balls_left']

In [ ]:
delivery_df

In [ ]:
delivery_df.shape

In [ ]:
def result(row):
    return 1 if row ['batting_team']== row['winner'] else 0

In [ ]:
delivery_df['result'] =delivery_df.apply(result,axis=1)

In [ ]:
delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [ ]:
final_df=delivery_df[['batting_team','bowling_team','city','runs_left','balls_left','wickets','total_runs_x','crr','rrr','result']]

In [ ]:
final_df=final_df.sample(final_df.shape[0])

In [ ]:
final_df.sample()

In [ ]:
final_df.isnull().sum()

In [ ]:
final_df.dropna(inplace=True)

In [ ]:
final_df =final_df[final_df['balls_left'] !=0 ]

In [ ]:
x = final_df.iloc[:, :-1]
y = final_df.iloc[:,-1]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
x_train

In [ ]:


from sklearn.preprocessing import OneHotEncoder

In [ ]:
from sklearn.compose import ColumnTransformer

trf =ColumnTransformer([
     ('trf',OneHotEncoder(sparse=False,drop='frist'),['batting_team','bowling_team','city'])
                         
]
,remainder='passthrough')

In [ ]:
!pip install transformers
from transformers import pipeline

In [ ]:
!pip install transformers==2.5.0

!pip install spacy-transformers==0.6.0

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import pipeline

In [ ]:
pipe=pipeline(steps=[
    ('step1',trf),
    ('step2',LogisticRegression(solver='liblinear'))
])

In [ ]:
pipe.fit(x_train,y_train)

In [ ]:
x_train.describe()

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)